In [1]:
import requests

from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
from pprint import pprint
import numpy as np
import re
import patsy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as sm
from statsmodels.tsa.arima_model import ARIMA
import warnings

import os
import sys
import json
import webbrowser
from json.decoder import JSONDecodeError
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets, metrics
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, confusion_matrix, classification_report
from sklearn.decomposition import PCA
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE, SMOTENC
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, AdaBoostRegressor, \
GradientBoostingClassifier, GradientBoostingRegressor, RandomForestClassifier
from sklearn import svm
from sklearn.tree import export_graphviz
from IPython import display

from joblib import dump, load

from matplotlib.colors import ListedColormap
cmap = ListedColormap(sns.color_palette(sns.diverging_palette(130, 260, center='light'),3))
import datetime
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import bokeh
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

from scipy.sparse import coo_matrix, hstack
from scipy import stats
from scipy import signal

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

sns.set(font_scale=1.5)



warnings.simplefilter('ignore')
pd.options.display.max_columns = 100

/anaconda3/envs/backup/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.



In [131]:
players = []

counter = 1
params = {
    'gender' : 'male',
    'order' : 'desc'
}

r_ = requests.get('https://www.fifaindex.com/players/fifa19_279/1/', params=params)
soup = BeautifulSoup(r_.text, 'html.parser')

while soup.findAll('tr'):

    for player in tqdm_notebook(soup.findAll('tr')[1:]):
        footballer = []

        try:
            footballer.append(player.find('td', attrs={'data-title': 'Name'}).text)
        except:
            footballer.append(np.nan)
        try:
            footballer.append(player.find('td', attrs={'data-title': 'Nationality'}).find('a')['title'])
        except:
            footballer.append(np.nan)
        try:
            footballer.append(player.find('td', attrs={'data-title': 'OVR / POT'}).find('span', attrs = {'class': True}).text)
        except:
            footballer.append(np.nan)

        positions =  player.find('td', attrs={'data-title': 'Preferred Positions'}).find_all('a')

        for i in range(3):

            try:
                footballer.append(positions[i]['title'])
            except:
                footballer.append(np.nan)

        try:
            footballer.append(player.find('td', attrs={'data-title': 'Age'}).text)
        except:
            footballer.append(np.nan)
        try:
            footballer.append(player.find('td', attrs={'data-title': 'Team'}).find('a')['title'])
        except:
            footballer.append(np.nan)

        players.append(footballer)
    counter += 1
    r_ = requests.get(f'https://www.fifaindex.com/players/fifa19_279/{counter}/', params=params)
    soup = BeautifulSoup(r_.text, 'html.parser')

In [133]:
cols = [
    'name',
    'nationality',
    'rating',
    'position1',
    'position2',
    'position3',
    'age',
    'team'
    
]

playaz = pd.DataFrame(players, columns=cols)

In [136]:
playaz.to_csv('playaz.csv', index=False)

In [129]:
soup.findAll('tr')[1].find('td', attrs={'data-title': 'OVR / POT'}).find('span', attrs = {'class': True}).text

'60'

In [132]:
players

[['Lionel Messi', 'Argentina', '94', 'CF', 'RW', 'ST', '31', 'FC Barcelona'],
 ['Cristiano Ronaldo', 'Portugal', '94', 'ST', 'LW', nan, '33', 'Juventus'],
 ['Neymar', 'Brazil', '92', 'LW', nan, nan, '26', 'Paris'],
 ['De Gea', 'Spain', '91', 'GK', nan, nan, '27', 'Manchester Utd'],
 ['Kevin De Bruyne',
  'Belgium',
  '91',
  'CAM',
  'CM',
  nan,
  '27',
  'Manchester City'],
 ['Eden Hazard', 'Belgium', '91', 'LW', 'CF', nan, '27', 'Chelsea'],
 ['Luka Modrić', 'Croatia', '91', 'CM', 'CDM', nan, '32', 'Real Madrid'],
 ['Luis Suárez', 'Uruguay', '91', 'ST', nan, nan, '31', 'FC Barcelona'],
 ['Sergio Ramos', 'Spain', '91', 'CB', nan, nan, '32', 'Real Madrid'],
 ['Jan Oblak', 'Slovenia', '90', 'GK', nan, nan, '25', 'Atlético Madrid'],
 ['Thibaut Courtois', 'Belgium', '90', 'GK', nan, nan, '26', 'Real Madrid'],
 ['Robert Lewandowski', 'Poland', '90', 'ST', nan, nan, '30', 'FC Bayern'],
 ['Toni Kroos', 'Germany', '90', 'CM', 'CDM', nan, '28', 'Real Madrid'],
 ['Diego Godín', 'Uruguay', '90',